In [6]:
import numpy as np
import time
import sys
import os

In [ ]:
"""
The notebook is a simple offline beamformer that can be used to generate
the gain pattern of 1 receiver element within the BEST phased array 
in H and E plane. Each receiver is composed of 16 dipole antennas 
which are combined using an analogue combiner.

Using the simulated gain pattern for each of the 16 dipoles, the gain pattern
for the receiver element can be obtained by summing the 
individual contributions of each dipole. The phase 
difference for each diploe antenna has to be 
taken into account before the summation.

"""

In [72]:
# List of constants
C = 299792458.0 # speed of light
F_C = 408000000 # center frequency
_LAMBDA = C / F_C # wavelength
N_DIPOLES = 16 # number of BEST dipoles to combine
N_CYLINDERS = 1 # Number of BEST cylinders to combine
N_THETA = 257 # Number of data points in the theta plane
N_PHI = 1024  # Number of data points in the phi plane
ELEMENT_ID = 1 # Determine which element within the array to use
NAME = "Combined Gain Pattern"

In [57]:
dx = _LAMBDA / 2 # distance between BEST dipoles
dy = 1 # distance between BEST cylinders

In [73]:
x = np.linspace(0, dx, N_DIPOLES)
y = np.linspace(0, dy, N_CYLINDERS)
xv, yv = np.meshgrid(x, y)

In [79]:
k0 = 2 * np.pi / _LAMBDA

theta = np.radians(np.linspace(0, 90, n_theta))
phi = np.radians(np.linspace(0, 360, n_phi))
kx = np.zeros((n_phi, n_theta))
ky = np.zeros((n_phi, n_theta))

for i in range(n_phis):
    for j in range(n_thetas):
        kx[i, j] = k0 * np.cos(phi[i]) * np.sin(theta[j])
        ky[i, j] = k0 * np.sin(phi[i]) * np.sin(theta[j])

[[0.         0.05246824 0.10493451 ... 8.55040381 8.55088672 8.55104769]
 [0.         0.05246725 0.10493253 ... 8.55024254 8.55072544 8.5508864 ]
 [0.         0.05246428 0.10492659 ... 8.54975872 8.55024159 8.55040255]
 ...
 [0.         0.05246428 0.10492659 ... 8.54975872 8.55024159 8.55040255]
 [0.         0.05246725 0.10493253 ... 8.55024254 8.55072544 8.5508864 ]
 [0.         0.05246824 0.10493451 ... 8.55040381 8.55088672 8.55104769]]


In [ ]:
# Load the elements gain pattern in H and V plane (to be loaded from Eman's simulation)
H = np.ndarray(shape=(N_THETA, N_PHI, N_DIPOLES))
E = np.ndarray(shape=(N_THETA, N_PHI, N_DIPOLES))

In [ ]:
# Loop over all the elements patters to compute average element pattern
for dipole in range(N_DIPOLES):
    # Compute total phase offset
    phase_offset = kx * xv[0, dipole] + ky * yv[0, dipole]
    center_shift = np.exp(-1j * phase_offset)

    # Phase shift the element pattern  (theta, phi, dipole)   
    H[:, :, i] *= center_shift
    E[:, :, i] *= center_shift

    
# Combine the phase-corrected elements pattern into a single beam
combined_H = np.sum(H, axis=2)
combined_E = np.sum(E, axis=2)

In [ ]:
# Visualise the radiation pattern of the combined element